In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, Date
import datetime as dt

# Part 1: Analyze and Explore the Climate Data

Use the SQLAlchemy create_engine() function to connect to your SQLite database.

In [ ]:
engine = create_engine('sqlite:///Resources/hawaii.sqlite')

#### Use the SQLAlchemy automap_base() function to reflect your tables into classes, and then save references to the classes named station and measurement.

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to the classes
Station = Base.classes.station
Measurement = Base.classes.measurement

#### Link Python to the database by creating a SQLAlchemy session.

In [ ]:
# Create a session to interact with the database
session = Session(engine)

#### Perform a precipitation analysis and then a station analysis by completing the steps in the following two subsections.
1. Find the most recent date in the dataset.

In [ ]:
most_recent_date = session.query(func.max(Measurement.date)).scalar()
most_recent_date

2. Using that date, get the previous 12 months of precipitation data by querying the previous 12 months of data.

In [ ]:
one_year_ago = dt.date.fromisoformat(most_recent_date) - dt.timedelta(days=365)
one_year_ago

In [ ]:
results = session.query(Measurement) \
    .filter(Measurement.date > one_year_ago) \
    .all()
results

3. Select only the "date" and "prcp" values.

In [ ]:
results = session.query(Measurement.date, Measurement.prcp) \
    .filter(Measurement.date > one_year_ago) \
    .all()
results

4. Load the query results into a Pandas DataFrame. Explicitly set the column names.

In [ ]:
df = pd.DataFrame(results)
df

5. Sort the DataFrame values by "date".

In [ ]:
date_df = df.sort_values('date')
date_df

6. Plot the results by using the DataFrame plot method

In [ ]:
import matplotlib.pyplot as plt

# Plot the precipitation data
date_df.plot(x='date', y='prcp', figsize=(5, 6))
plt.xlabel('Date')
plt.ylabel('Precipitation (inches)')
plt.title('Precipitation Analysis')
plt.legend(loc='upper center')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Print summary statistics
print(df['prcp'].describe())

## Stational Analysis

1. Design a query to calculate the total number of stations in the dataset.

In [ ]:
station_count = session.query(func.count(Station.station)).scalar()
station_count

2. Design a query to find the most-active stations (that is, the stations that have the most rows). To do so, complete the following steps:

- List the stations and observation counts in descending order.

- Answer the following question: which station id has the greatest number of observations?


In [ ]:
station_activity = session.query(Measurement.station, func.count(Measurement.station))\
                          .group_by(Measurement.station)\
                          .order_by(func.count(Measurement.station).desc())\
                          .all()

In [ ]:
for station, count in station_activity:
    print(f"Station: {station}, Observation Count: {count}")

In [ ]:
most_active_station = station_activity[0][0]
print(f"The station with the greatest number of observations is: {most_active_station}")

3. Design a query that calculates the lowest, highest, and average temperatures that filters on the most-active station id found in the previous query.

In [ ]:
# Query to calculate temperature statistics for the most active station
temperature_stats = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs))\
                           .filter(Measurement.station == most_active_station)\
                           .all()

# Print the temperature statistics
lowest_temp = temperature_stats[0][0]
highest_temp = temperature_stats[0][1]
avg_temp = temperature_stats[0][2]
print(f"For the most active station ({most_active_station}):")
print(f"Lowest Temperature: {lowest_temp}")
print(f"Highest Temperature: {highest_temp}")
print(f"Average Temperature: {avg_temp}")


4. Design a query to get the previous 12 months of temperature observation (TOBS) data. To do so, complete the following steps:

- Filter by the station that has the greatest number of observations.

- Query the previous 12 months of TOBS data for that station.

- Plot the results as a histogram with bins=12.

In [ ]:
# Query to get the previous 12 months of TOBS data for the most active station
tobs_data = session.query(Measurement.tobs)\
                  .filter(Measurement.station == most_active_station)\
                  .filter(Measurement.date >= one_year_ago)\
                  .filter(Measurement.date <= most_recent_date)\
                  .all()

# Convert the query results to a DataFrame
tobs_df = pd.DataFrame(tobs_data, columns=['TOBS'])

# Plot the results as a histogram
tobs_df.plot.hist(bins=12, legend=False)
plt.xlabel('Temperature (°F)')
plt.ylabel('Frequency')
plt.title('Temperature Observation (TOBS)')
plt.show()


5. Close your session.

In [ ]:
session.close()

# Part 2: Design Your Climate App
Now that you’ve completed your initial analysis, you’ll design a Flask API based on the queries that you just developed. To do so, use Flask to create your routes as follows:

1. /

- Start at the homepage.

- List all the available routes.

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def home():
    return (
        "Welcome to the Climate App API!<br/>"
        "Available Routes:<br/>"
        "/api/v1.0/precipitation<br/>"
        "/api/v1.0/stations<br/>"
        "/api/v1.0/tobs<br/>"
        "/api/v1.0/&lt;start&gt;<br/>"
        "/api/v1.0/&lt;start&gt;/&lt;end&gt;<br/>"
    )

if __name__ == "__main__":
    app.run(debug=True)


2. /api/v1.0/precipitation

- Convert the query results from your precipitation analysis (i.e. retrieve only the last 12 months of data) to a dictionary using date as the key and prcp as the value.

- Return the JSON representation of your dictionary.

In [ ]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    session = Session(engine)
    results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_year_ago).all()

    session.close()


3. /api/v1.0/stations

- Return a JSON list of stations from the dataset.

In [ ]:
def stations():
    session = Session(engine)

    results = session.query(Station.station).all()

    session.close()

    station_list = [station[0] for station in results]

    return jsonify(station_list)

if __name__ == "__main__":
    app.run(debug=True)


4. /api/v1.0/tobs

- Query the dates and temperature observations of the most-active station for the previous year of data.

- Return a JSON list of temperature observations for the previous year.

5. /api/v1.0/<start> and /api/v1.0/<start>/<end>

Return a JSON list of the minimum temperature, the average temperature, and the maximum temperature for a specified start or start-end range.

For a specified start, calculate TMIN, TAVG, and TMAX for all the dates greater than or equal to the start date.

For a specified start date and end date, calculate TMIN, TAVG, and TMAX for the dates from the start date to the end date, inclusive.